In [1]:
# Model Configuration
CONV_TYPE = "fuse"  # "ori_aegnn" or "fuse"
MODEL_NAME = "500.pth"  # Model filename
MODEL_PATH = "/home/benio/Documents/GNNBenchmark/ResultsRecognition/Recognition_EVGNN_on_ncaltech1/TrainedModels"  # Path to trained models

# Dataset Configuration
DATASET = "ncaltech"  # "ncars" or "ncaltech"
DATASET_PATHS = {
    "ncars": r"/home/benio/Documents/Datasets/NCars",
    "ncaltech": r"/home/benio/Documents/Datasets/NCaltech"
}

# Evaluation Configuration
NUM_SAMPLES = 10  # Number of test samples to evaluate
EVENTS_PER_SAMPLE = 5000  # Number of events to process per sample for async metrics
N_EVENTS_SAMPLE = 10000  # Number of events to sample per recording

# Output Configuration
OUTPUT_DIR = "../results/async_test_results2"  # Directory to save results (same location as models)

# Graph Construction Parameters
RADIUS = 5.0  # Radius for graph construction
MAX_NUM_NEIGHBORS = 32  # Max neighbors in graph
MAX_DT = 66000  # Max time difference for edges
BETA = 0.1e-4  # Time normalization beta
USE_MICROSECONDS = False  # Model was trained with normalized timestamps!

# Device Configuration
DEVICE = "cuda"

In [2]:
import os
import sys
import time
import gc
import torch
import numpy as np
import json
from copy import deepcopy
from torch_geometric.data import Data
from torch_geometric.nn.pool import radius_graph
from torch_geometric.transforms import Cartesian
from tqdm import tqdm

from src.Models.CleanEvGNN.GraphRes_Base import GraphRes as EvGNN
from Models.CleanEvGNN.asyncronous import make_model_asynchronous, reset_async_module
from Models.CleanEvGNN.asyncronous_aegnn import make_model_asynchronous as make_model_asynchronous_aegnn
from Models.CleanEvGNN.asyncronous_aegnn import reset_async_module as reset_async_module_aegnn
from Datasets.ncars import NCars
from Datasets.ncaltech101 import NCaltech
from Datasets.batching import BatchManager
from Models.utils import normalize_time, sub_sampling

# Optional: Power consumption tracking (only available on Linux with AIPowerMeter)
try:
    from Benchmarks.ModelTester import ModelTester
    POWER_TRACKING_AVAILABLE = True
    print("✓ Power consumption tracking available")
except ImportError:
    POWER_TRACKING_AVAILABLE = False
    print("⚠️  Power consumption tracking not available (AIPowerMeter not installed)")



/home/benio/miniconda3/envs/gnn-benchmarking/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Power consumption tracking available


## Load Datset and Model

In [3]:


dataset_path = DATASET_PATHS[DATASET]

if DATASET == 'ncars':
    dataset_obj = NCars(root=dataset_path)
    num_classes = len(NCars.get_info().classes)
    image_size = NCars.get_info().image_size
elif DATASET == 'ncaltech':
    dataset_obj = NCaltech(root=dataset_path)
    num_classes = len(NCaltech.get_info().classes)
    image_size = NCaltech.get_info().image_size
else:
    raise ValueError(f"Unknown dataset: {DATASET}")

dataset_obj.process(modes=["test"])
num_test_samples = dataset_obj.get_mode_length("test")

x

📂 Processing folder: mayfly


mayfly: 100%|██████████| 4/4 [00:00<00:00, 21732.15it/s]



📂 Processing folder: starfish


starfish: 100%|██████████| 10/10 [00:00<00:00, 55333.83it/s]



📂 Processing folder: pizza


...

lobster: 100%|██████████| 5/5 [00:00<00:00, 33341.05it/s]



📂 Processing folder: Motorbikes


Motorbikes: 100%|██████████| 81/81 [00:00<00:00, 143896.07it/s]



📂 Processing folder: sea_horse


sea_horse: 100%|██████████| 7/7 [00:00<00:00, 49678.73it/s]


In [4]:
device = torch.device(DEVICE)
img_shape = (image_size[1], image_size[0])

model = EvGNN(
    input_shape = torch.tensor([*img_shape, 3]),
    dataset=DATASET,
    num_outputs=num_classes,
    conv_type=CONV_TYPE,
    distill=False
).to(device)

model_path = os.path.join(MODEL_PATH, MODEL_NAME)
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model not found: {model_path}")

model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

GraphRes(
  (fuse1): MyConvBNReLU(1, 16)
  (fuse2): MyConvBNReLU(16, 32)
  (fuse3): MyConvBNReLU(32, 32)
  (fuse4): MyConvBNReLU(32, 32)
  (pool): MaxPoolingX(voxel_size=tensor([16., 16.], device='cuda:0'), size=180)
  (fc): qLinear(
    (lin): Linear(in_features=5760, out_features=101, bias=False)
    (obs_in): MinMaxObserver(min_val=0.0, max_val=6.4146342277526855)
    (obs_out): MinMaxObserver(min_val=-63.36524200439453, max_val=17.986724853515625)
    (obs_w): MinMaxObserver(min_val=-0.7886757254600525, max_val=0.149931401014328)
  )
  (drop): Dropout(p=0.0, inplace=False)
)

In [5]:
def transform_sample(sample, device):
    """Apply preprocessing to match original AEGNN dataset pre_transform"""
    sample = sample.to(device)

    # Normalize polarity
    sample.x = torch.where(sample.x == -1., 0., sample.x)

    # Subsample events
    sample = sub_sampling(sample, n_samples=N_EVENTS_SAMPLE, sub_sample=True)

    if USE_MICROSECONDS:
        sample.pos[:, 2] = torch.round(sample.pos[:, 2] * 1e6)
    else:
        sample.pos[:, 2] = normalize_time(sample.pos[:, 2], beta=BETA)

    # Build graph with standard radius_graph
    sample.edge_index = radius_graph(sample.pos, r=RADIUS, max_num_neighbors=MAX_NUM_NEIGHBORS)

    # Add edge attributes
    edge_attr_fn = Cartesian(cat=False, max_value=RADIUS)
    sample.edge_attr = edge_attr_fn(sample).edge_attr

    return sample

## Setup Async Model and test

In [6]:

test_loader = BatchManager(dataset=dataset_obj, batch_size=1, mode="test")
num_samples = min(NUM_SAMPLES, num_test_samples)

edge_attributes = Cartesian(norm=True, cat=False)

model_for_async = deepcopy(model)

if CONV_TYPE == 'ori_aegnn':
    async_model = make_model_asynchronous_aegnn(model_for_async, r=RADIUS, edge_attributes=edge_attributes,
                                                log_flops=False, log_runtime=False)
    reset_async_fn = reset_async_module_aegnn
else:
    async_model = make_model_asynchronous(model_for_async, r=RADIUS, max_num_neighbors=MAX_NUM_NEIGHBORS,
                                          max_dt=MAX_DT, edge_attributes=edge_attributes,
                                          log_flops=False, log_runtime=False)
    reset_async_fn = reset_async_module



## Run Full Async Evaluation

In [7]:

# Initialize power consumption tracking
model_tester = None
if POWER_TRACKING_AVAILABLE and sys.platform == "linux":
    power_output_dir = os.path.join(OUTPUT_DIR, "power_consumption")
    os.makedirs(power_output_dir, exist_ok=True)
    model_tester = ModelTester(
        results_path=power_output_dir,
        model=async_model
    )
    print("✓ Power consumption tracking initialized")
elif POWER_TRACKING_AVAILABLE:
    print("⚠️  Power tracking only works on Linux, skipping measurement")

per_event_latencies = []
all_predictions = []
all_targets = []
predictions_per_event = []
successful_samples = 0
failed_samples = 0

gc.collect()
if device.type == 'cuda':
    torch.cuda.empty_cache()

events_to_process = min(EVENTS_PER_SAMPLE, N_EVENTS_SAMPLE)

##
##  Training loop with power measurement
##


# Use context manager for power measurement (like in training script)
if model_tester is not None:
    print("testing")
    # Wrap evaluation loop with power measurement
    with model_tester:
        for i in tqdm(range(num_samples), desc="Async inference"):
            try:
                sample = next(test_loader)
                sample = transform_sample(sample, device)
                target_class = sample.y.item()
                all_targets.append(target_class)

                reset_async_fn(async_model)

                num_events = min(sample.num_nodes, events_to_process)
                sample_predictions = []

                with torch.no_grad():
                    for event_idx in range(num_events):
                        event_new = Data(
                            x=sample.x[event_idx:event_idx+1],
                            pos=sample.pos[event_idx:event_idx+1, :3],
                            batch=torch.zeros(1, dtype=torch.long),
                            edge_index=torch.empty((2, 0), dtype=torch.long),
                            edge_attr=torch.empty((0, 3), dtype=torch.float)
                        ).to(device)

                        event_start = time.perf_counter()
                        output = async_model(event_new)
                        latency = (time.perf_counter() - event_start) * 1000
                        per_event_latencies.append(latency)

                        pred = torch.argmax(output, dim=-1).item()
                        sample_predictions.append(pred)

                        if event_idx == num_events - 1:
                            all_predictions.append(pred)

                predictions_per_event.append(sample_predictions)
                successful_samples += 1

            except (IndexError, RuntimeError) as e:
                print(e)
                failed_samples += 1
                if len(all_targets) > len(all_predictions):
                    all_predictions.append(0)
                predictions_per_event.append([0])
                continue


##
## Training loop without power measurement
##

else:
    # No power measurement
    pass


✓ Power consumption tracking initialized
testing
power meter not avaible: /home/ntirel/libs/wattmeter_rapid_omegawatt/wattmetre-read  does not exist
GPU power will be measured with nvidia


Async inference: 100%|██████████| 10/10 [02:29<00:00, 15.00s/it]


Done with measuring


## Calculate Metrics


In [8]:

# Compute metrics
predictions = np.array(all_predictions)
targets = np.array(all_targets)
accuracy_final = (predictions == targets).mean()


# Compute latency statistics
latency_mean = float(np.mean(per_event_latencies))
latency_std = float(np.std(per_event_latencies))

# Compute accuracy evolution over events
max_events = max(len(preds) for preds in predictions_per_event)
accuracy_evolution = []
for event_idx in range(max_events):
    correct = sum(1 for sample_idx, sample_preds in enumerate(predictions_per_event)
                 if event_idx < len(sample_preds) and sample_preds[event_idx] == all_targets[sample_idx])
    total = sum(1 for sample_preds in predictions_per_event if event_idx < len(sample_preds))
    accuracy_evolution.append(correct / total if total > 0 else 0.0)

# Count correct/incorrect predictions
num_correct = int((predictions == targets).sum())
num_incorrect = successful_samples - num_correct

# Print results
print("\n" + "="*70)
print("ASYNCHRONOUS EVALUATION RESULTS")
print("="*70)
print(f"Accuracy: {accuracy_final:.4f} ({num_correct}/{successful_samples} correct)")
print(f"Latency:  {latency_mean:.4f} ± {latency_std:.4f} ms")
print(f"Samples:  {successful_samples}/{num_samples} successful")
print("="*70)

# Power consumption summary
power_summary = None
if model_tester is not None:
    print("\n" + "="*70)
    print("POWER CONSUMPTION")
    print("="*70)
    try:
        if model_tester._power_consumption_results_exist():
            model_tester.print_power_consumption()
            power_summary = model_tester.summarize_power_consumption()
            print("✓ Power consumption data collected")
        else:
            print("⚠️  No power consumption data found")
    except Exception as e:
        print(f"⚠️  Could not retrieve power consumption: {e}")
    print("="*70)

# Save accuracy evolution to CSV
os.makedirs(OUTPUT_DIR, exist_ok=True)
model_name = os.path.splitext(MODEL_NAME)[0]
base_name = f"{model_name}_{DATASET}"

csv_path = os.path.join(OUTPUT_DIR, f"{base_name}_accuracy_evolution.csv")
with open(csv_path, 'w') as f:
    f.write("event_index,accuracy\n")
    for idx, acc in enumerate(accuracy_evolution):
        f.write(f"{idx},{acc:.6f}\n")

# Save metrics to JSON
json_path = os.path.join(OUTPUT_DIR, f"{base_name}_metrics.json")
all_metrics = {
    'model': MODEL_NAME,
    'dataset': DATASET,
    'accuracy': float(accuracy_final),
    'latency_mean_ms': latency_mean,
    'latency_std_ms': latency_std,
    'num_correct': num_correct,
    'num_incorrect': num_incorrect,
    'successful_samples': successful_samples,
    'failed_samples': failed_samples
}

# Add power consumption metrics if available
if power_summary is not None:
    all_metrics['power_consumption'] = power_summary

with open(json_path, 'w') as f:
    json.dump(all_metrics, f, indent=2)



ASYNCHRONOUS EVALUATION RESULTS
Accuracy: 0.3000 (3/10 correct)
Latency:  2.9112 ± 1.0994 ms
Samples:  10/10 successful

POWER CONSUMPTION
============================================ EXPERIMENT SUMMARY ============================================

Experiment duration:  148.09908413887024 seconds.  Start: 2026-01-20 17:21:07.293959  end 2026-01-20 17:23:35.393043
ENERGY CONSUMPTION: 
on the cpu

CPU time usage of your experiment: 148.40956218004231 seconds
RAM consumption not available. Your usage was  1.3GiB among which 916.3MiB is unique to your experiment (ie. USS memory)
Total CPU consumption: 28.857043261945805 joules, your experiment consumption:  26.201706198542208 joules
total intel power:  3145.549999691258 joules
total psys power:  -148.09908413887024 joules


GPU
nvidia total consumption: 3524.102981171607 joules, your consumption:  3175.6112851513712 joules
Max memory used:
    gpu: 0 : 212.0MiB
Average GPU usage:
    gpu: 0: 16.568 %
Attributable usage per GPU
    gpu: 0:

In [9]:
# # Analyze predictions - check if model predicted varied classes
# print("\n" + "="*70)
# print("PREDICTION ANALYSIS")
# print("="*70)
#
# # Count unique predictions
# unique_preds, pred_counts = np.unique(predictions, return_counts=True)
# print(f"\nTotal predictions: {len(predictions)}")
# print(f"Unique classes predicted: {len(unique_preds)}/{num_classes}")
# print(f"\nPrediction distribution:")
# for cls, count in zip(unique_preds, pred_counts):
#     percentage = (count / len(predictions)) * 100
#     print(f"  Class {int(cls)}: {count:3d} times ({percentage:5.1f}%)")
#
# # Check if model is stuck predicting one class
# if len(unique_preds) == 1:
#     print(f"\n❌ WARNING: Model predicted ONLY class {int(unique_preds[0])}!")
#     print(f"   The model is stuck and not working properly.")
# elif len(unique_preds) < num_classes / 2:
#     print(f"\n⚠️  Model only uses {len(unique_preds)} out of {num_classes} classes")
#     print(f"   This may indicate a problem with the model or data.")
# else:
#     print(f"\n✓ Model uses {len(unique_preds)} different classes (good diversity)")
#
# # Show ground truth distribution for comparison
# print(f"\nGround truth distribution:")
# unique_targets, target_counts = np.unique(targets, return_counts=True)
# for cls, count in zip(unique_targets, target_counts):
#     percentage = (count / len(targets)) * 100
#     print(f"  Class {int(cls)}: {count:3d} times ({percentage:5.1f}%)")
#
# # Per-class accuracy
# print(f"\nPer-class accuracy:")
# for cls in range(num_classes):
#     cls_mask = targets == cls
#     if cls_mask.sum() > 0:
#         cls_acc = (predictions[cls_mask] == targets[cls_mask]).mean()
#         cls_count = cls_mask.sum()
#         correct = int(cls_acc * cls_count)
#         print(f"  Class {cls}: {cls_acc:.4f} ({correct}/{cls_count} correct)")
#
# print("="*70)
